In [83]:
import pandas as pd
import requests


dfsolar = pd.read_csv('datos_solares.csv').drop_duplicates().drop(0).reset_index(drop=True)
dfsolar.columns = ['Ciudad','Horas de sol','Irradiación']
dfsolar['Horas de sol'] = dfsolar['Horas de sol'].astype(float)*1000

#Irradiación solar [kWh/m2·día]
# Convierte los valores de la columna a float
dfsolar['Irradiación'] = dfsolar['Irradiación'].str.replace(' kWh/m2día', '')
dfsolar['Irradiación'] = dfsolar['Irradiación'].str.replace(',', '.')

dfsolar.to_csv('datos_solares2.csv', index=False)

### DATOS HORARIOS

# URL del archivo de texto
def sacar_datos_horarios(ciudad):    
    url = f'https://cdn.mitma.gob.es/portal-web-drupal/salidapuestasol/2024/{ciudad}-2024.txt'

    # Descargar contenido
    response = requests.get(url)
    response.encoding = 'ISO-8859-1'
    content = response.text

    # Dividir en líneas y omitir las filas de encabezado y pie
    lines = content.splitlines()[4:6] + content.splitlines()[7:]  # Quitar encabezado

    # Crear listas de listas para estructurar datos manualmente,
    data = []

    for line in lines:
        # Trata de dividir las líneas de manera personalizada
        parts = line.split() 
        data.append(parts)

    # Columnas de cabeceras
    columnas = ['Dia', 'Ene_Ort', 'Ene_Ocas', 'Feb_Ort', 'Feb_Ocas', 'Mar_Ort', 'Mar_Ocas', 'Apr_Ort', 'Apr_Ocas', 
                'May_Ort', 'May_Ocas', 'Jun_Ort', 'Jun_Ocas', 'Jul_Ort', 'Jul_Ocas', 'Aug_Ort', 'Aug_Ocas',
            'Sep_Ort', 'Sep_Ocas', 'Oct_Ort', 'Oct_Ocas', 'Nov_Ort', 'Nov_Ocas', 'Dec_Ort', 'Dec_Ocas']

    # Crear DataFrame
    df = pd.DataFrame(data).iloc[:-4].dropna(axis=1, how='all')
    df.columns = columnas
    df = df.iloc[2:].reset_index(drop=True)

    def format_time(value):
        if isinstance(value, str):
            # Asegura que los valores tengan cuatro caracteres
            if len(value) == 3:
                value = '0' + value
            # Inserta ':' antes del penúltimo carácter
            value = value[:-2] + ':' + value[-2:]
        return value

    # Aplica la función a todas las columnas excepto la primera
    df.iloc[:, 1:] = df.iloc[:, 1:].applymap(format_time)

    # Vamos a transformar los datos en formato hora

    for col in df.columns[1:]:  # Ignora la primera columna ('Dia')
        # Convertir a Timedelta
        df[col] = pd.to_timedelta(df[col] + ':00')

    # Lista de meses abreviados
    meses = ["Ene", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    # Crear el nuevo DataFrame con la columna 'Dia'
    nuevo_df = pd.DataFrame()
    nuevo_df['Dia'] = df['Dia']
    nuevo_df['Ciudad'] = ciudad

    # Calcular la duración de cada mes
    for mes in meses:
        nuevo_df[mes] = (df[f'{mes}_Ocas'] - df[f'{mes}_Ort']).apply(lambda x: x.total_seconds() / 3600)

    return nuevo_df

df_caceres = sacar_datos_horarios('Sevilla')
df_caceres


,Dia,Ciudad,Ene,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1,Sevilla,9.633333,10.333333,11.383333,12.633333,13.750000,14.566667,14.700000,14.033333,12.950000,11.783333,10.616667,9.766667
1,2,Sevilla,9.650000,10.366667,11.433333,12.683333,13.800000,14.600000,14.683333,14.016667,12.916667,11.733333,10.566667,9.750000
2,3,Sevilla,9.650000,10.400000,11.466667,12.716667,13.833333,14.616667,14.666667,13.983333,12.883333,11.716667,10.533333,9.733333
3,4,Sevilla,9.666667,10.433333,11.516667,12.750000,13.866667,14.616667,14.650000,13.950000,12.850000,11.666667,10.500000,9.716667
4,5,Sevilla,9.683333,10.466667,11.550000,12.800000,13.900000,14.633333,14.650000,13.916667,12.800000,11.633333,10.466667,9.716667
5,6,Sevilla,9.700000,10.500000,11.583333,12.816667,13.933333,14.650000,14.633333,13.883333,12.766667,11.583333,10.450000,9.700000
6,7,Sevilla,9.716667,10.533333,11.633333,12.866667,13.950000,14.666667,14.633333,13.850000,12.716667,11.550000,10.416667,9.683333
7,8,Sevilla,9.716667,10.566667,11.666667,12.900000,13.983333,14.683333,14.600000,13.816667,12.700000,11.500000,10.383333,9.666667
8,9,Sevilla,9.733333,10.600000,11.716667,12.950000,14.016667,14.683333,14.600000,13.800000,12.650000,11.466667,10.350000,9.650000
9,10,Sevilla,9.766667,10.633333,11.750000,12.983333,14.050000,14.700000,14.583333,13.766667,12.616667,11.416667,10.316667,9.633333
